In [1]:
!pip install jupyter_dash

In [2]:
# Import required libraries
import pandas as pd
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-2-74714d25a243>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-74714d25a243>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
# Read the airline data into pandas dataframe and define the min and max payloads.
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [14]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                    dcc.Dropdown(id='site_dropdown',
                                        options=[{'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                        value='ALL',
                                        placeholder="Select a Launch Site here",
                                        searchable=True
                                        ),
                                        html.Br(),
                                    html.Div(dcc.Graph(id='success-pie-chart')),
                                            html.Br(),
                                    
                                            html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0:'0kg', 1000:'1000kg',2000:'2000kg', 3000:'3000kg',
                                                4000:'4000kg', 5000:'5000kg',6000:'6000kg', 7000:'7000kg', 8000:'8000kg',
                                                9000:'9000kg',10000:'10000kg'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Callback decorator
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site_dropdown', component_property='value'))

def get_pie_chart(site_dropdown):
    if site_dropdown == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names='Launch Site', title='Total Success Launches By site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site_dropdown].groupby(['Launch Site', 'class']). \
                size().reset_index(name='class count')
        title = f"Total Success Launches for site {site_dropdown}"
        filtered_df = filtered_df
        fig = px.pie(filtered_df, values='class count', names='class', title=title)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),
      Input(component_id="payload-slider", component_property='value')]
)

def get_scatter_plot(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        low, high=payload_slider
        spacex_df1=spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
        fig=px.scatter(spacex_df1, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                hover_data=['Booster Version'],
                    title='Correlation between Payload and Success for All Sites')
        return fig
    else:
        sites_df=spacex_df1[spacex_df1['Launch Site'] == site_dropdown]
        fig=px.scatter(sites_df, x='Payload Mass (kg)', y='class',
                        color='Booster Version Category', hover_data=['Booster Version'],
                        title='Success Launch for' + site_dropdown + 'Site With Respect to Payload Mass (kg)')
        return fig



In [15]:
# Run the app
app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,
          dev_tools_hot_reload =True, threaded=True)